<img width="200" style="float:left" 
     src="https://upload.wikimedia.org/wikipedia/commons/f/f3/Apache_Spark_logo.svg" />

<img style="float:left" src="https://storage.googleapis.com/kaggle-competitions/kaggle/3136/logos/header.png" />

# Sections
* [Description](#0)
* [1. Setup](#1)
  * [1.1 Start Hadoop](#1.1)  
  * [1.2 Search for Spark Installation](#1.2)
  * [1.3 Create SparkSession](#1.3)
* [2. Lab](#2)
  * [2.1 Check Lab Files](#2.1)
* [3. Data Preparation](#3)
  * [3.1 Data Cleansing](#3.1)
  * [3.1 Feature Engineering](#3.1)
* [4. Model Training](#4)
* [5. Model Evaluation](#5)
* [6. Model Selection](#6)
* [7. Model Persistence](#7)
* [8. Model Loading](#8)
* [9. Challenge](#9)
* [10. TearDown](#10)
  * [10.1 Stop Hadoop](#10.1)

<a id='0'></a>
## Description
<p>
<div>The goal for this lab is:</div>
<ul>    
    <li>Practice Spark's Machine Learning API</li>
</ul>
</p>

The goal is to create a machine learning model to predict if a passenger would survive or not, therefore is a classification problem.

<a id='1'></a>
## 1. Setup

Since we are going to process data stored from HDFS let's start the service

<a id='1.1'></a>
### 1.1 Start Hadoop

Start Hadoop

Open a terminal and execute
```sh
hadoop-start.sh
```

<a id='1.2'></a>
### 1.2 Search for Spark Installation 
This step is only needed in our course environment; other Spark environments you might see out there, might not need this statement.

In [424]:
import findspark
findspark.init()

In order to improve how data is displayed, I'll setup Pandas accordingly.

In [425]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

<a id='1.3'></a>
### 1.3 Create SparkSession

By setting this environment variable we can include extra libraries in our Spark cluster.<br/>

In [426]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = ' pyspark-shell'

Time to create the SparkSession which we'll use to send our Spark code:

In [427]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName("Titanic - Analytics - MLlib")
    .config("spark.sql.warehouse.dir","hdfs://localhost:9000/warehouse")
    .enableHiveSupport()
    .getOrCreate())

<a id='2'></a>
## 2. Lab

<a id='2.1'></a>
### 2.1 Check Lab Files

In order to complete this lab you need to previosly upload the datasets into HDFS.<br/>

Check you have the data ready in HDFS

http://localhost:50070/explorer.html#/datalake/raw/kaggle/titanic/

<a id='2.2'></a>
### 2.2 Data Information
We have the following information about the dataset:
<table style="float:left">
<tbody>
<tr><th><b>Variable</b></th><th><b>Definition</b></th><th><b>Key</b></th></tr>
<tr>
<td>survival</td>
<td>Survival</td>
<td>0 = No, 1 = Yes</td>
</tr>
<tr>
<td>pclass</td>
<td>Ticket class</td>
<td>1 = 1st, 2 = 2nd, 3 = 3rd</td>
</tr>
<tr>
<td>sex</td>
<td>Sex</td>
<td></td>
</tr>
<tr>
<td>Age</td>
<td>Age in years</td>
<td></td>
</tr>
<tr>
<td>sibsp</td>
<td># of siblings / spouses aboard the Titanic</td>
<td></td>
</tr>
<tr>
<td>parch</td>
<td># of parents / children aboard the Titanic</td>
<td></td>
</tr>
<tr>
<td>ticket</td>
<td>Ticket number</td>
<td></td>
</tr>
<tr>
<td>fare</td>
<td>Passenger fare</td>
<td></td>
</tr>
<tr>
<td>cabin</td>
<td>Cabin number</td>
<td></td>
</tr>
<tr>
<td>embarked</td>
<td>Port of Embarkation</td>
<td>C = Cherbourg, Q = Queenstown, S = Southampton</td>
</tr>
</tbody>
</table>

<a id='3'></a>
## 3. Data Preparation

<a id='3.1'></a>
### 3.1 Data Cleansing

In [428]:
titanic_raw = (spark.read
                    .option("inferSchema", "true")
                    .option('header', 'true')
                    .csv("hdfs://localhost:9000/datalake/raw/kaggle/titanic/")
                    .cache())

In [429]:
titanic_raw.limit(5).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [430]:
titanic_raw.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



Let's do some *Exploratory Data Analysis* to understand our data a bit better

In [431]:
passengers_count = titanic_raw.count()
print (f"Total number of passenger: {passengers_count}")

Total number of passenger: 891


**Summary of data**:

In [432]:
titanic_raw.summary().toPandas()

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,25%,223,0,2,None,None,20.0,0,0,19996.0,7.8958,None,None
5,50%,446,0,3,None,None,28.0,0,0,236171.0,14.4542,None,None
6,75%,669,1,3,None,None,38.0,1,0,347743.0,31.0,None,None
7,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


Let's analize the number of **passengers who survived**:

In [433]:
titanic_raw.groupBy("Survived").count().toPandas()

,Survived,count
0,1,342
1,0,549


342 out of the 891 passengers survived.

Let's dig into **specific information about survivors** by exploring some more data.

The survival rate can be determined by different features of the dataset such as *sex*, *port of embarcation*, *age*, ...

Let's analyze the **survival rate using feature sex**:

In [434]:
titanic_raw.groupBy("Sex","Survived").count().toPandas()

,Sex,Survived,count
0,male,0,468
1,female,1,233
2,female,0,81
3,male,1,109


Even though the number of males is greater than the number of females in the ship, **females' survival rate is twice of the males'**.
Let's analyze the **survival rate using feature pclass**:

In [435]:
titanic_raw.groupBy("Pclass","Survived").count().toPandas()

,Pclass,Survived,count
0,1,0,80
1,3,1,119
2,1,1,136
3,2,1,87
4,2,0,97
5,3,0,372


It's clear that people in *pclass #1* had much more priority than people in *pclass #3*; **even though the number of passengers in pclass #3 was higher, the survival rate was very low.**

In [436]:
titanic_df = titanic_raw

### Null Values

Let's **check if there are null values** that we need to remove before moving forward.

There are **two ways of coming up with the number of null values**:

In [437]:
from pyspark.sql.functions import isnull, when, count, col

# Option 1
titanic_df.select([count(when(isnull(c), c)).alias(c) for c in titanic_df.columns]).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,0,0,0,0,177,0,0,0,0,687,2


The other way is summarizing

In [438]:
# Option 2
titanic_df.summary().toPandas()

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,25%,223,0,2,None,None,20.0,0,0,19996.0,7.8958,None,None
5,50%,446,0,3,None,None,28.0,0,0,236171.0,14.4542,None,None
6,75%,669,1,3,None,None,38.0,1,0,347743.0,31.0,None,None
7,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


There are 3 features with missing data.

**Cabin** feature has 687 (891 - 204) null values.

**Embarked** feature has 2 (891 - 889) null values.

**Age** feature has 177 (891 - 714) null values.

Let's decide what to do

#### Cabin
Since there are so many missing values we have to get rid of this feature

In [439]:
titanic_df = titanic_df.drop("Cabin")

#### Embarked
Embarked feature has only two missining values. Let's check values within Embarked



In [440]:
titanic_df.groupBy("Embarked").count().toPandas()

,Embarked,count
0,Q,77
1,None,2
2,C,168
3,S,644


Majority Passengers boarded from "S". We can impute these with "S"

In [441]:
titanic_df = titanic_df.na.fill({"Embarked" : 'S'})

#### Age
This is going to be a tricky one :) in some scenarios you can replace a missing value with the mean,median, mode value of the dataset.There is a functionality available is Spark fot his job called <a href="https://spark.apache.org/docs/latest/ml-features.html#imputer">Imputer</a>

But this approach won't work here... **it might happen that you end up assignment a 4 years old kid a average age value of 29**.

We'll try to come up with a representative number based on other features. The **Name feature seems to be a good one** to calculate an average age for missing values.

**Names start with a word like Mr or Mrs**; calculating the average age for each group, will be closer to the real missing value... let's go for it:

In [442]:
from pyspark.sql.functions import regexp_extract
titanic_df = titanic_df.withColumn("Initial",regexp_extract(col("Name"),"([A-Za-z]+)\.",1))

The regular expression "([A-Za-z]+)\\." extracts the words we're looking for: **text containing characters between A-Z or a-z and followed by a .(dot)**.

In [443]:
titanic_df.limit(5).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Initial
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Mr


In [444]:
titanic_df.select("Initial").distinct().sort("Initial").toPandas()

,Initial
0,Capt
1,Col
2,Countess
3,Don
4,Dr
5,Jonkheer
6,Lady
7,Major
8,Master
9,Miss


There are some misspelled Initials like Mlle or Mme that stand for Miss. I will replace them with Miss and same thing for other values.

In [445]:
titanic_df = titanic_df.replace(
               ['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [446]:
titanic_df.select("Initial").distinct().toPandas()

,Initial
0,Miss
1,Other
2,Master
3,Mr
4,Mrs


Let's impute missing values in age feature based on average age of Initials
1. Calculate the average value based on the initials and create a DataFrame with those values:

In [447]:
from pyspark.sql.functions import round

avg_age_df = (titanic_df.groupby('Initial').avg('Age')
                        .withColumnRenamed("avg(Age)","Age"))
avg_age_df.toPandas()

,Initial,Age
0,Miss,21.860000
1,Other,45.888889
2,Master,4.574167
3,Mr,32.739609
4,Mrs,35.981818


2. Let's create a temporary DF removing the Age column in those records/rows without a age (Age field with Null value):

In [448]:
titanic_df_noage = titanic_df.where(col("Age").isNull()).drop("Age")
titanic_df_noage.limit(1).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,Initial
0,6,0,3,"Moran, Mr. James",male,0,0,330877,8.4583,Q,Mr


3. Add an Age field to the previous DataFrame by joining it with the average age DataFrame:

In [449]:
titanic_df_noage_with_avg = titanic_df_noage.join(avg_age_df, "Initial")
titanic_df_noage_with_avg.limit(1).toPandas()

,Initial,PassengerId,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,Age
0,Mr,6,0,3,"Moran, Mr. James",male,0,0,330877,8.4583,Q,32.739609


4. Take the original titanic_df, **keep records/rows with a non-null age** and add the previous DataFrame to the result via an union transformation:

In [450]:
titanic_df_fixed = (titanic_df.where(col("Age").isNotNull())
                      .unionByName(titanic_df_noage_with_avg))

titanic_df_fixed.where(col("Age").isNull()).count()

0

In [451]:
titanic_df = titanic_df_fixed

#### Family_size and Alone
Let's create a couple of features called **Family_size** and **Alone**, which might bring some insights on survival rate and the size of the families.

**Family_size** is the total number of *parch (parents/children)* and *sibsp (siblings/spouses)* per row. **Alone** will be a flag set when the size of the family equals to 0.

In [452]:
titanic_df = titanic_df.withColumn("Family_Size",col('SibSp')+col('Parch'))

In [453]:
titanic_df.groupBy("Family_Size").count().toPandas()

,Family_Size,count
0,1,161
1,6,12
2,3,29
3,5,22
4,4,15
5,7,6
6,2,102
7,0,537
8,10,7


In [454]:
from pyspark.sql.functions import lit
titanic_df = titanic_df.withColumn("Alone",when(titanic_df["Family_Size"] == 0, 1).otherwise(lit(0)))

In [455]:
titanic_df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Embarked',
 'Initial',
 'Family_Size',
 'Alone']

<a id='3.2'></a>
### 3.2. Feature Engineering
It's time to convert data into a suitable format for machine learning algorithms.<br/> 
First let's get rid of columns with unique values that don't contribute to a persons survival probability

In [456]:
from pyspark.sql.functions import countDistinct

titanic_df.select([countDistinct(c).alias(c) for c in titanic_df.columns]).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Initial,Family_Size,Alone
0,891,2,3,891,2,92,7,7,681,248,3,5,9,2


In [457]:
titanic_df = titanic_df.drop("PassengerId","Name","Ticket","Initial")

In [458]:
titanic_df.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- Family_Size: integer (nullable = true)
 |-- Alone: integer (nullable = false)



Let's cast all numerical values to doubles

In [459]:
titanic_df = titanic_df.select(col('Survived').cast('double'),
                              col('Pclass').cast('double'),
                              col('Sex'),
                              col('Age').cast('double'),
                              col('SibSp').cast('double'),
                              col('Parch').cast('double'),
                              col('Fare').cast('double'),
                              col('Embarked'),
                              col('Family_Size').cast('double'),
                              col('Alone').cast('double')
                             )

In [460]:
titanic_df.printSchema()

root
 |-- Survived: double (nullable = true)
 |-- Pclass: double (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- Family_Size: double (nullable = true)
 |-- Alone: double (nullable = false)



#### 3.2.2 Feature Transformation

We need to **translate the values in string columns into  numerical values**.

In order to do so, we are going to *encode* categorical values using:<br/>

[StringIndexer](https://spark.apache.org/docs/latest/ml-features#stringindexer) <br/>
[OneHotEncoder](https://spark.apache.org/docs/latest/ml-features#onehotencoder)

In [604]:
label_column = "Survived"

categoricalCols = [field for (field, dataType) in titanic_df.dtypes if ((dataType == "string") & (field != label_column))]
numericCols = [field for (field, dataType) in titanic_df.dtypes if ((dataType == "double") & (field != label_column))]

print (f"categorical columns: {categoricalCols}")
print (f"numerical columns: {numericCols}")

categorical columns: ['Sex', 'Embarked']
numerical columns: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Family_Size', 'Alone']


In [605]:
indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

print (f"StringIndexer column names: {indexOutputCols}")
print (f"OHE column names: {oheOutputCols}")

StringIndexer column names: ['SexIndex', 'EmbarkedIndex']
OHE column names: ['SexOHE', 'EmbarkedOHE']


In [606]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=indexOutputCols, handleInvalid="skip")

oheEncoder = OneHotEncoder(inputCols=indexOutputCols,outputCols=oheOutputCols)

Checking StringIndexer

In [607]:
temp_df = stringIndexer.fit(titanic_df).transform(titanic_df)
temp_df.toPandas()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family_Size,Alone,SexIndex,EmbarkedIndex
0,0.0,3.0,male,22.000000,1.0,0.0,7.2500,S,1.0,0.0,0.0,0.0
1,1.0,1.0,female,38.000000,1.0,0.0,71.2833,C,1.0,0.0,1.0,1.0
2,1.0,3.0,female,26.000000,0.0,0.0,7.9250,S,0.0,1.0,1.0,0.0
3,1.0,1.0,female,35.000000,1.0,0.0,53.1000,S,1.0,0.0,1.0,0.0
4,0.0,3.0,male,35.000000,0.0,0.0,8.0500,S,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,3.0,male,32.739609,0.0,0.0,7.2292,C,0.0,1.0,0.0,1.0
887,0.0,3.0,female,21.860000,8.0,2.0,69.5500,S,10.0,0.0,1.0,0.0
888,0.0,3.0,male,32.739609,0.0,0.0,9.5000,S,0.0,1.0,0.0,0.0
889,0.0,3.0,male,32.739609,0.0,0.0,7.8958,S,0.0,1.0,0.0,0.0


Checking OneHotEncoder

In [608]:
oheEncoder.fit(temp_df).transform(temp_df).toPandas()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family_Size,Alone,SexIndex,EmbarkedIndex,SexOHE,EmbarkedOHE
0,0.0,3.0,male,22.000000,1.0,0.0,7.2500,S,1.0,0.0,0.0,0.0,(1.0),"(1.0, 0.0)"
1,1.0,1.0,female,38.000000,1.0,0.0,71.2833,C,1.0,0.0,1.0,1.0,(0.0),"(0.0, 1.0)"
2,1.0,3.0,female,26.000000,0.0,0.0,7.9250,S,0.0,1.0,1.0,0.0,(0.0),"(1.0, 0.0)"
3,1.0,1.0,female,35.000000,1.0,0.0,53.1000,S,1.0,0.0,1.0,0.0,(0.0),"(1.0, 0.0)"
4,0.0,3.0,male,35.000000,0.0,0.0,8.0500,S,0.0,1.0,0.0,0.0,(1.0),"(1.0, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,3.0,male,32.739609,0.0,0.0,7.2292,C,0.0,1.0,0.0,1.0,(1.0),"(0.0, 1.0)"
887,0.0,3.0,female,21.860000,8.0,2.0,69.5500,S,10.0,0.0,1.0,0.0,(0.0),"(1.0, 0.0)"
888,0.0,3.0,male,32.739609,0.0,0.0,9.5000,S,0.0,1.0,0.0,0.0,(1.0),"(1.0, 0.0)"
889,0.0,3.0,male,32.739609,0.0,0.0,7.8958,S,0.0,1.0,0.0,0.0,(1.0),"(1.0, 0.0)"


In [609]:
assemblerInputs = oheOutputCols + numericCols
print("Feature columns: ",assemblerInputs)

Feature columns:  ['SexOHE', 'EmbarkedOHE', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Family_Size', 'Alone']


#### 3.2.3 Feature Assembling

It's finally time to **assemble the features in one single vector**, which is what the algorithm will expect, by using something called [VectorAssembler](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=vectorassembler#pyspark.ml.feature.VectorAssembler).

As the **"Survived" variable** is the one we want to predict, **all the other variables** will be considered to build the **list with required features**:

In [610]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol="features")

In [611]:
from pyspark.ml import Pipeline

test_pipeline = Pipeline(stages = [stringIndexer, oheEncoder, vecAssembler])
features_df = test_pipeline.fit(titanic_df).transform(titanic_df)
features_df.limit(2).toPandas()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family_Size,Alone,SexIndex,EmbarkedIndex,SexOHE,EmbarkedOHE,features
0,0.0,3.0,male,22.0,1.0,0.0,7.2500,S,1.0,0.0,0.0,0.0,(1.0),"(1.0, 0.0)","[1.0, 1.0, 0.0, 3.0, 22.0, 1.0, 0.0, 7.25, 1.0, 0.0]"
1,1.0,1.0,female,38.0,1.0,0.0,71.2833,C,1.0,0.0,1.0,1.0,(0.0),"(0.0, 1.0)","[0.0, 0.0, 1.0, 1.0, 38.0, 1.0, 0.0, 71.2833, 1.0, 0.0]"


<a id='4'></a>
## 4. Model Training

Here is the list of few classification algorithms from Spark ML we are going to try:

<ul>
<li>LogisticRegression</li>
<li>DecisionTreeClassifier</li>
<li>RandomForestClassifier</li>
<li>Gradient-boosted tree classifier</li>
<li>NaiveBayes</li>
<li>Linear Support Vector Machine</li>
</ul>

There are some points in the machine learning workflow were randomness takes place, for example during the sets splitting and some ML algorithms like RandomForest.

In order to make our experiments reproducible and get always the same results with the same data no matter how many times we execute our code, we are going to use a seed.

The seed is tipically a prime number.

In [469]:
seed=11

In [470]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import LinearSVC

lr = LogisticRegression(labelCol="Survived", featuresCol="features")
dt = DecisionTreeClassifier(labelCol="Survived", featuresCol="features",seed=seed)
rf = RandomForestClassifier(labelCol="Survived", featuresCol="features",maxDepth=10,seed=seed)
gbt = GBTClassifier(labelCol="Survived", featuresCol="features",maxIter=10,seed=seed)
nb = NaiveBayes(labelCol="Survived", featuresCol="features")
svm = LinearSVC(labelCol="Survived", featuresCol="features")

classifiers = [lr,dt,rf,gbt,nb,svm]
classifiers

[LogisticRegression_736f603cfed5,
 DecisionTreeClassifier_b98d90f62063,
 RandomForestClassifier_c9f44b280ba7,
 GBTClassifier_f013dffef4f8,
 NaiveBayes_dab2b073294b,
 LinearSVC_c64921c4df51]

Let's create a pipeline for every classifier

In [471]:
from pyspark.ml import Pipeline

def create_pipeline(classifier):
    return Pipeline(stages = [stringIndexer, oheEncoder, vecAssembler, classifier])

pipelines = [create_pipeline(classifier) for classifier in classifiers]
pipelines

[Pipeline_d88fd93ba4d9,
 Pipeline_c5c623d90e8f,
 Pipeline_2a93df5d98b7,
 Pipeline_0accee1f22bd,
 Pipeline_eeaf8bf72963,
 Pipeline_e600ee3fd391]

<a id='5'></a>
## 5. Model Evaluation
We're going to evaluate our classification model by using [MulticlassClassificationEvaluator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html)

We're going to use the accuray metric.

In [472]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Survived",  metricName="accuracy")

<a id='6'></a>
## 6. Model Selection

Now that the data is all set, let's split it into training and test. We can use a 80-20 ratio

In [473]:
(trainingData, testData) = titanic_df.randomSplit([0.8,0.2],seed=seed)

It's a good practice to keep the same distribution of 0's and 1's in the training set, and specially critical in umbalanced/skew datasets. This is called **Stratified Train-Test Split**

In [474]:
(trainingData1, testData1) = titanic_df.where("Survived=0").randomSplit([0.8,0.2],seed=seed)
(trainingData2, testData2) = titanic_df.where("Survived=1").randomSplit([0.8,0.2],seed=seed)

trainingData_st = trainingData1.unionByName(trainingData2)
testData_st = testData1.unionByName(testData2)

Let's train all the classifiers

In [475]:
models = [pipeline.fit(trainingData) for pipeline in pipelines]
models

[PipelineModel_744d3f901846,
 PipelineModel_8a22b4eccf7e,
 PipelineModel_9156b3647097,
 PipelineModel_a5120e7c84bb,
 PipelineModel_c10e6847f073,
 PipelineModel_5c314a02d1ab]

Let's evaluate all the models

In [476]:
names = []
values = [] 
for model in models:
     
    names.append(type(model.stages[-1]).__name__) # the algorithm is the last stage in the pipeline
    values.append(accuracy)

data = {'name':names,'accuracy':values,'model':models}
df = pd.DataFrame(data)
df.sort_values(by=['accuracy'], inplace=True, ascending=False)  
df

,name,accuracy,model
1,DecisionTreeClassificationModel,0.832432,PipelineModel_8a22b4eccf7e
3,GBTClassificationModel,0.816216,PipelineModel_a5120e7c84bb
2,RandomForestClassificationModel,0.800000,PipelineModel_9156b3647097
5,LinearSVCModel,0.778378,PipelineModel_5c314a02d1ab
0,LogisticRegressionModel,0.772973,PipelineModel_744d3f901846
4,NaiveBayesModel,0.691892,PipelineModel_c10e6847f073


The best model is **RandomForestClassificationModel** (for professor)

For me, the best model appears to be **DecisionTreeClassificationModel** at 0.832432

Second best model is **GBTClassificationModel** at 0.816216

In [477]:
best_model=df.iloc[0]['model']

Confusion matrix

In [478]:
best_model.transform(testData).groupby("Survived").pivot("prediction").count().toPandas()

,Survived,0.0,1.0
0,0.0,106,9
1,1.0,22,48


<a id='7'></a>
## 7. Model Persistence
Spark provides functionality to save the model/pipeline so that we can use it later for inference (batch or streaming)

In [479]:
modelPath = "hdfs://localhost:9000/model-registry/titanic-survival-classifier"
best_model.write().overwrite().save(modelPath)

Check directory contents

http://localhost:50070/explorer.html#/model-registry/titanic-survival-classifier/


<a id='8'></a>
## 8. Model Loading

In [480]:
from pyspark.ml import PipelineModel
savedModel = PipelineModel.load(modelPath)

In [723]:
predictions = savedModel.transform(testData)
predictions.select("features", "Survived", "prediction").limit(200).toPandas()

,features,Survived,prediction
0,"[1.0, 1.0, 0.0, 1.0, 19.0, 3.0, 2.0, 263.0, 5.0, 0.0]",0.0,0.0
1,"[1.0, 1.0, 0.0, 1.0, 21.0, 0.0, 1.0, 77.2875, 1.0, 0.0]",0.0,0.0
2,"[1.0, 0.0, 1.0, 1.0, 28.0, 1.0, 0.0, 82.1708, 1.0, 0.0]",0.0,0.0
3,"[1.0, 1.0, 0.0, 1.0, 29.0, 0.0, 0.0, 30.0, 0.0, 1.0]",0.0,0.0
4,"[1.0, 1.0, 0.0, 1.0, 29.0, 1.0, 0.0, 66.6, 1.0, 0.0]",0.0,0.0
...,...,...,...
180,"(0.0, 0.0, 0.0, 3.0, 21.86, 0.0, 0.0, 7.75, 0.0, 1.0)",1.0,1.0
181,"(0.0, 0.0, 0.0, 3.0, 21.86, 0.0, 0.0, 7.7875, 0.0, 1.0)",1.0,1.0
182,"(0.0, 0.0, 0.0, 3.0, 21.86, 0.0, 0.0, 7.8792, 0.0, 1.0)",1.0,1.0
183,"(0.0, 0.0, 0.0, 3.0, 21.86, 1.0, 0.0, 24.15, 1.0, 0.0)",1.0,1.0


<a id='9'></a>
## 10. Challenge

¿Can you improve this model?

Try to find and remove outliers.

Try new features or drop existing features.

Try different feature transformations. 

Try different feature scalers.

Try different algorithms and parameters.

Try cross-validation or train-validation split with grid parameters

In [713]:
indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

print (f"StringIndexer column names: {indexOutputCols}")
print (f"OHE column names: {oheOutputCols}")

StringIndexer column names: ['SexIndex', 'EmbarkedIndex']
OHE column names: ['SexOHE', 'EmbarkedOHE']


In [714]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=indexOutputCols, handleInvalid="skip")

oheEncoder = OneHotEncoder(inputCols=indexOutputCols,outputCols=oheOutputCols)

In [715]:
assemblerInputs = oheOutputCols + numericCols
print("Feature columns: ",assemblerInputs)

Feature columns:  ['SexOHE', 'EmbarkedOHE', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Family_Size', 'Alone']


In [716]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# UDF for converting column type from vector to double type
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

# Iterating over columns to be scaled
for i in ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Family_Size', 'Alone']:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline2 = Pipeline(stages=[assembler, scaler])

In [717]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol="features")

In [718]:
dt = DecisionTreeClassifier(labelCol="Survived", featuresCol="features",seed=seed)
rf = RandomForestClassifier(labelCol="Survived", featuresCol="features",seed=seed)

In [730]:
#Pipeline
#pipeline = Pipeline(stages = [stringIndexer, oheEncoder, vecAssembler, pipeline2, savedModel])
pipeline = Pipeline(stages = [pipeline2, savedModel])

In [731]:
(trainingData, testData) = titanic_df.randomSplit([0.8,0.2],seed=seed)

In [732]:
#Cross Validation
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [2, 5, 10, 20, 30]) \
    .addGrid(dt.maxBins, [20, 40, 60]) \
    .build()

paramGrid_rf = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [2, 5, 10, 20, 30]) \
    .addGrid(rf.maxBins, [20, 40, 60]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)  # use 3+ folds in practice

In [740]:
fitModel = crossval.fit(trainingData)
BestModel = fitModel.bestModel

prediction_df = BestModel.transform(testData)
accuracy = evaluator.evaluate(prediction_df)
accuracy

0.8324324324324325

In [751]:
for i in featureImportances.argsort()[-5:][::-1]:
    print(assemblerInputs[i])

SexOHE
SibSp
Age
Alone
Family_Size


In [752]:
indexOutputCols = [x + "Index" for x in ['Sex']]
oheOutputCols = [x + "OHE" for x in ['Sex']]

print (f"StringIndexer column names: {indexOutputCols}")
print (f"OHE column names: {oheOutputCols}")

StringIndexer column names: ['SexIndex']
OHE column names: ['SexOHE']


In [753]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

stringIndexer = StringIndexer(inputCols=['Sex'], outputCols=indexOutputCols, handleInvalid="skip")

oheEncoder = OneHotEncoder(inputCols=indexOutputCols,outputCols=oheOutputCols)

In [774]:
#SexOHE SibSp Age Alone Family_Size
assemblerInputs_imp = ['SexOHE', 'Age', 'Family_Size']

In [775]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=assemblerInputs_imp,outputCol="features")

In [778]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# UDF for converting column type from vector to double type
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

# Iterating over columns to be scaled
for i in ["Age","Family_Size"]:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline2 = Pipeline(stages=[assembler, scaler])

In [779]:
#Pipeline
pipeline = Pipeline(stages = [stringIndexer, oheEncoder, vecAssembler, pipeline2, dt])

In [780]:
#Cross Validation
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid_rf = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [2, 5, 10]) \
    .addGrid(rf.maxBins, [20, 60]) \
    .build()

paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [2, 5, 10, 20, 30]) \
    .addGrid(dt.maxBins, [10, 20, 40, 60]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=4)  # use 3+ folds in practice

In [781]:
cvModel = crossval.fit(trainingData)
BestModel = cvModel.bestModel

predictions = BestModel.transform(testData)
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % (accuracy))

Accuracy = 0.821622


In [782]:
modelPath = "hdfs://localhost:9000/model-registry/titanic-survival-classifier_tuned"
BestModel.write().overwrite().save(modelPath)

<a id='10'></a>
## 10. Tear Down

Once we complete the the lab we can stop all the services

<a id='10.1'></a>
### 10.1 Stop Hadoop

Stops Hadoop
Open a terminal and execute
```sh
hadoop-stop.sh
```